In [3]:
import scanpy as sc

from scprint import scPrint
from scprint.tasks import Embedder

from scdataloader import Preprocessor
from scdataloader.utils import load_genes, translate

import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier

from scdataloader.utils import get_descendants
from sklearn.metrics import f1_score

import matplotlib.pyplot as plt
from scib_metrics.benchmark import Benchmarker, BioConservation, BatchCorrection

from collections import Counter

%load_ext autoreload
%autoreload 2

import torch
torch.set_float32_matmul_precision('medium')

→ connected lamindb: jkobject/scprint2


In [ ]:
m = torch.load('../../xzp23r4p.ckpt', map_location=torch.device('cpu'))

In [ ]:
model = scPrint.load_from_checkpoint(
    '../../xzp23r4p.ckpt', precpt_gene_emb=None, classes=m['hyper_parameters']['label_counts'], 
)#strict=False)
    #transformer="normal")

FYI: scPrint is not attached to a `Trainer`.


## Loading the data

In [ ]:
adata = sc.read(
    "data/pancreas_atlas.h5ad",
    backup_url="https://figshare.com/ndownloader/files/24539828",
)


In [5]:
adata = sc.read(
    "data/human_brain_smartv4.h5ad",
    backup_url="https://datasets.cellxgene.cziscience.com/f156b755-78c7-4928-917a-be138c153b85.h5ad",
)


  0%|          | 0.00/2.47G [00:00<?, ?B/s]

In [8]:
adata.X.sum()

66897050000.0

In [ ]:
preprocessor = Preprocessor(force_preprocess=True, skip_validate=True,
                            do_postp=False, min_valid_genes_id=5000, min_dataset_size=64)
nadata = preprocessor(adata.copy())

Dropping layers:  KeysView(Layers with keys: exon, intron)
checking raw counts
removed 0 non primary cells, 49417 renamining
filtered out 0 cells, 49417 renamining
Removed 0 genes.
startin QC
Seeing 10181 outliers (20.60% of total dataset):
done
AnnData object with n_obs × n_vars = 49417 × 70704
    obs: 'suspension_type', 'cluster', 'class', 'subclass', 'sex_ontology_term_id', 'region', 'cortical_layer', 'cell_type_accession', 'cell_type_alias', 'cell_type_alt_alias', 'cell_type_designation', 'donor_id', 'outlier_call', 'outlier_type', 'tissue_ontology_term_id', 'disease_ontology_term_id', 'assay_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'cell_type_ontology_term_id', 'Specimen ID', 'sample_tissue_type', 'development_stage_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'cause_of_death', 'PMI (hr)', 'Tissue (RIN)', 'Hemisphere Sampled', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'develo

AnnData object with n_obs × n_vars = 49417 × 35793
    obs: 'suspension_type', 'cluster', 'class', 'subclass', 'sex_ontology_term_id', 'region', 'cortical_layer', 'cell_type_accession', 'cell_type_alias', 'cell_type_alt_alias', 'cell_type_designation', 'donor_id', 'outlier_call', 'outlier_type', 'tissue_ontology_term_id', 'disease_ontology_term_id', 'assay_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'cell_type_ontology_term_id', 'Specimen ID', 'sample_tissue_type', 'development_stage_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'cause_of_death', 'PMI (hr)', 'Tissue (RIN)', 'Hemisphere Sampled', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length', 'feature_type'
    uns: 'citation', 'cortical_layer_colors', 'region_colors', 'schema_reference', 'schema

In [ ]:
ln.from_adata(nadata)

# training

In [17]:
from scdataloader import DataModule

In [18]:
gene_emb = '../data/main/gene_embeddings.parquet'
datamodule = DataModule(
    collection_name="scPRINT-V2 test", #some, all, preprocessed dataset, all no zhang, 
    gene_embeddings=gene_emb,
    clss_to_weight=[],
    metacell_mode=0,
    clss_to_predict=[],
    hierarchical_clss=[],
    organisms=["NCBITaxon:9606", "NCBITaxon:10090"],
    how="random expr",
    max_len=10_000,
    add_zero_genes=0,
    # how much more you will see the most present vs less present category
    weight_scaler=0,
    batch_size=1,
    num_workers=3,
    prefetch_factor=3,
    n_samples_per_epoch=2000,
    validation_split=0.05,
    do_gene_pos='../data/main/biomart_pos.parquet',
    pin_memory=True,
    test_split=0.05)
testfiles = datamodule.setup()

! no run & transform got linked, call `ln.track()` & re-run
! run input wasn't tracked, call `ln.track()` and re-run
won't do any check but we recommend to have your dataset coming from local storage


/home/ml4ig1/Documents code/scDataLoader/scdataloader/utils.py:400: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  organismdf = pd.concat(organismdf)


100.0% are aligned
seeing a string: loading gene positions as biomart parquet file


KeyError: 'organism_ontology_term_id'

# testing

In [ ]:
# denoising without


In [ ]:
# denoising

In [ ]:
# gene network inference without

In [ ]:
# gene network inference